In [1]:
import tensorflow as tf

## Linear Layer -- Dimensionsreduktion/-erhöhung
$$y = f(x \cdot W)$$

In [2]:
# Modellinputs
BATCH_SIZE = 2
NUM_INPUTS = 10
INPUT_DIM = 3
x = tf.random.normal(shape=(BATCH_SIZE, NUM_INPUTS, INPUT_DIM))
print("Dimension des Inputs:", x.shape)

# Unser einfaches Modell mit 1 Linear Layer
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=5)
])
model.build(input_shape=x.shape)
print("Matrixdimension des Linear Layers:", model.layers[0].weights[0].shape)

# Der Linear Layer wird immer auf die letzte Dimension (3. Dim) angewendet 
# Der Linear Layer kann parallel für alle Inputs (2. Dim) berechnet werden
y = model(x)
print("Dimension des Outputs:", y.shape)

Dimension des Inputs: (2, 10, 3)
Matrixdimension des Linear Layers: (3, 5)
Dimension des Outputs: (2, 10, 5)


## Recurrent Layer -- Informationen aus dem vorigen Zeitschritt verwenden
$$s_t = f(s_{t-1} \cdot R + x_t \cdot W)$$

In [5]:
# wir benutzen "x" aus dem vorigen Beispiel
print("Dimension des Inputs:", x.shape)

# Unser einfaches Modell mit 1 Linear Layer
model = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(units=5, return_sequences=True)
])
model.build(input_shape=x.shape)

# Ein RNN hat einen Linear Layer, um die Input Dimension (hier 3) zu erhöhen (hier 5).
# Der rekurrente Kernel (hier 5x5 Matrix) wird danach angewendet.
# Die Inputs (2. Dim) ist die Zeitdimension t einer Sequenz/Zeitreihe.
y = model(x)
print("Dimension des Outputs:", y.shape)

Dimension des Inputs: (2, 10, 3)
Dimension des Outputs: (2, 10, 5)


## Convolution Layer -- Aggregiere benachbarte Inputs

In [6]:
# wir benutzen "x" aus dem vorigen Beispiel
print("Dimension des Inputs:", x.shape)

# Unser einfaches Modell mit 1 Linear Layer
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=1, kernel_size=2)
])
model.build(input_shape=x.shape)

# Der "Kernel" gleitet hier über "kernel_size=2" benachbarte Inputs und aggregiert diese.
y = model(x)
print("Dimension des Outputs:", y.shape)

Dimension des Inputs: (2, 10, 3)
Dimension des Outputs: (2, 9, 1)


In [74]:
# Die trainierbaren "Filter"-Gewichte
model.layers[0].weights[0]

<tf.Variable 'conv1d_14/kernel:0' shape=(2, 3, 1) dtype=float32, numpy=
array([[[-0.5877724 ],
        [ 0.37192458],
        [ 0.01260459]],

       [[ 0.1083079 ],
        [ 0.5347014 ],
        [-0.2614751 ]]], dtype=float32)>

## Attention Mechanism

In [10]:
# wir benutzen "x" aus dem vorigen Beispiel
print("Dimension des Inputs:", x.shape)

dim = 5.

# Projektionslayer (Wir benutzen hier die Keras Functional API)
Q = tf.keras.layers.Dense(units=dim)(x)
K = tf.keras.layers.Dense(units=dim)(x)
V = tf.keras.layers.Dense(units=dim)(x)

# Die "Energy" weights W=Q*K.T
W = tf.einsum('bij,bkj->bik', Q, K)
# W = tf.einsum('bij,bkj->bik', x, x)
print("Energy:", W.shape)

# Attention Matrix
scaler = 1. / tf.sqrt(dim)
A = tf.nn.softmax(W * scaler, axis=2)
print("Attention:", A.shape)

# Multipliziere mit V
Y = tf.einsum('bij,bjk->bik', A, V)
print("Dimension des Outputs:", Y.shape)

Dimension des Inputs: (2, 10, 3)
Energy: (2, 10, 10)
Attention: (2, 10, 10)
Dimension des Outputs: (2, 10, 5)
